In [27]:
import numpy as np
import numpy.typing as npt
import pytest
import stim
from graphix import Circuit, Pattern, command
from graphix.fundamentals import Plane
from graphix.noise_models.depolarising_noise_model import DepolarisingNoiseModel
from graphix.noise_models.noise_model import NoiseModel
from graphix.random_objects import rand_circuit
from graphix.sim.base_backend import (
    FixedBranchSelector,
    RandomBranchSelector,
)
from graphix.simulator import DefaultMeasureMethod
from graphix.states import BasicStates
from numpy.random import PCG64, Generator
from veriphix.client import Client, Secrets
from veriphix.trappifiedCanvas import TrappifiedCanvas

from gospel.brickwork_state_transpiler import (
    ConstructionOrder,
    generate_random_pauli_pattern,
    get_bipartite_coloring,
)
from gospel.scripts import compare_backend_results
from gospel.stim_pauli_preprocessing import (
    StimBackend,
    cut_pattern,
    preprocess_pauli,
    simulate_pauli,
)

In [28]:
import numpy as np
from numpy.random import PCG64, Generator
from typing import List, Dict

# Assuming these classes and functions are imported properly
# from your library
# from your_library import Secrets, Client, get_bipartite_coloring, TrappifiedCanvas, StimBackend, DepolarisingNoiseModel, generate_random_pauli_pattern, command

# Initialization
fx_bg = PCG64(42)
jumps = 5
n_iterations = 100  # Number of test iterations

# Define separate outcome tables for Canonical and Deviant
test_outcome_table_canonical: List[Dict] = []
test_outcome_table_deviant: List[Dict] = []

# Loop over Construction Orders
for order in (ConstructionOrder.Canonical, ConstructionOrder.Deviant):
    rng = Generator(fx_bg.jumped(jumps))  # Use the jumped rng
    pattern = generate_random_pauli_pattern(
        nqubits=2, nlayers=2, order=order, rng=rng
    )

    # Add measurement commands to the output nodes
    for onode in pattern.output_nodes:
        pattern.add(command.M(node=onode))

    # Initialize secrets and client
    secrets = Secrets(r=False, a=False, theta=False)
    client = Client(pattern=pattern, secrets=secrets)
    
    # Get bipartite coloring and create test runs
    colours = get_bipartite_coloring(pattern)
    test_runs = client.create_test_runs(manual_colouring=colours)

    # Define backend and noise model
    backend = StimBackend()
    noise_model = DepolarisingNoiseModel(entanglement_error_prob=0.5)

    n_failures = 0

    # Choose the correct outcome table based on order
    if order == ConstructionOrder.Canonical:
        test_outcome_table = test_outcome_table_canonical
    else:
        test_outcome_table = test_outcome_table_deviant

    print(f"Running {n_iterations} iterations for order: {order}", flush=True)

    # Run the test iterations
    for i in range(n_iterations):
        # Select a random test run
        run = TrappifiedCanvas(test_runs[rng.integers(len(test_runs))], rng=rng)

        # Delegate the test run to the client
        trap_outcomes = client.delegate_test_run(backend=backend, run=run, noise_model=noise_model)
        
        
        # Create a result dictionary (trap -> outcome)
        result = {
            tuple(trap): outcome for trap, outcome in zip(run.traps_list, trap_outcomes)
        }

        # Append the result to the appropriate test outcome table
        test_outcome_table.append(result)

        # Print pass/fail based on the sum of the trap outcomes
        if sum(trap_outcomes) != 0:
            n_failures += 1
            print(f"Iteration {i+1}: ❌ Failed trap round", flush=True)
        else:
            print(f"Iteration {i+1}: ✅ Trap round passed", flush=True)

    # Final report after completing the test rounds
    print(f"Final result for {order}: {n_failures}/{n_iterations} failed rounds", flush=True)
    print("-" * 50, flush=True)

    # Uncomment this line if you want to assert no failures occurred
    # assert n_failures == 0, f"Test failed: {n_failures} trap rounds detected noise."

# Now you have two separate outcome tables:
# test_outcome_table_canonical for Canonical order
# test_outcome_table_deviant for Deviant order


Running 100 iterations for order: ConstructionOrder.Canonical
Iteration 1: ❌ Failed trap round
Iteration 2: ❌ Failed trap round
Iteration 3: ❌ Failed trap round
Iteration 4: ❌ Failed trap round
Iteration 5: ❌ Failed trap round
Iteration 6: ❌ Failed trap round
Iteration 7: ❌ Failed trap round
Iteration 8: ❌ Failed trap round
Iteration 9: ❌ Failed trap round
Iteration 10: ❌ Failed trap round
Iteration 11: ❌ Failed trap round
Iteration 12: ❌ Failed trap round
Iteration 13: ❌ Failed trap round
Iteration 14: ❌ Failed trap round
Iteration 15: ❌ Failed trap round
Iteration 16: ❌ Failed trap round
Iteration 17: ❌ Failed trap round
Iteration 18: ❌ Failed trap round
Iteration 19: ❌ Failed trap round
Iteration 20: ❌ Failed trap round
Iteration 21: ❌ Failed trap round
Iteration 22: ❌ Failed trap round
Iteration 23: ❌ Failed trap round
Iteration 24: ❌ Failed trap round
Iteration 25: ❌ Failed trap round
Iteration 26: ❌ Failed trap round
Iteration 27: ❌ Failed trap round
Iteration 28: ❌ Failed trap r

In [29]:
print(len(test_outcome_table_canonical))
occurences = {}
occurences_one = {}

for results in test_outcome_table_canonical:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba_can = {q: occurences_one[q] / occurences[q] for q in occurences}
print(failure_proba_can)

100
{(1,): 0.5434782608695652, (2,): 0.391304347826087, (5,): 0.5434782608695652, (6,): 0.43478260869565216, (9,): 0.34782608695652173, (10,): 0.4782608695652174, (13,): 0.5434782608695652, (14,): 0.43478260869565216, (17,): 0.45652173913043476, (0,): 0.5, (3,): 0.4444444444444444, (4,): 0.5370370370370371, (7,): 0.46296296296296297, (8,): 0.6851851851851852, (11,): 0.4444444444444444, (12,): 0.5555555555555556, (15,): 0.3888888888888889, (16,): 0.46296296296296297}


In [30]:
print(len(test_outcome_table_deviant))
occurences = {}
occurences_one = {}

for results in test_outcome_table_deviant:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba_dev = {q: occurences_one[q] / occurences[q] for q in occurences}
print(failure_proba_dev)

100
{(1,): 0.6086956521739131, (2,): 0.41304347826086957, (5,): 0.5652173913043478, (6,): 0.5, (9,): 0.5217391304347826, (10,): 0.41304347826086957, (13,): 0.45652173913043476, (14,): 0.5217391304347826, (17,): 0.5, (0,): 0.6111111111111112, (3,): 0.4444444444444444, (4,): 0.46296296296296297, (7,): 0.6111111111111112, (8,): 0.5925925925925926, (11,): 0.5370370370370371, (12,): 0.5740740740740741, (15,): 0.48148148148148145, (16,): 0.42592592592592593}


In [10]:
#Working on the equations now

In [31]:
import numpy as np
import sympy as sp


def generate_qubit_edge_matrix_with_unknowns(n, m):
    assert n % 2 == 0, "The number of rows (n) must be even."

    qubits = {}  # Mapping from (i, j) to qubit index
    edges = {}   # Mapping from edge (start, end) to edge index
    edge_index = 0
    qubit_index = 0

    # Assign an index to each qubit (i, j)
    for i in range(n):
        for j in range(m):
            qubits[(i, j)] = qubit_index
            qubit_index += 1

    # Collect all edges and assign them an index
    for i in range(n):
        for j in range(m - 1):
            edge = ((i, j), (i, j + 1))  # Horizontal edge
            edges[edge] = edge_index
            edge_index += 1

    for i in range(n - 1):
        for j in range(m):
            if ((j + 1) % 8 == 3 and (i + 1) % 2 != 0):  # Column j ≡ 3 (mod 8) and odd row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge = ((i, j + 1), (i + 1, j + 1))
                    edges[edge] = edge_index
                    edge_index += 1
                    edge = ((i, j + 3), (i + 1, j + 3))
                    edges[edge] = edge_index
                    edge_index += 1
            if ((j + 1) % 8 == 7 and (i + 1) % 2 == 0):  # Column j ≡ 7 (mod 8) and even row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge = ((i, j + 1), (i + 1, j + 1))
                    edges[edge] = edge_index
                    edge_index += 1
                    edge = ((i, j + 3), (i + 1, j + 3))
                    edges[edge] = edge_index
                    edge_index += 1

    # Create the symbolic matrix (qubits × edges)
    matrix = np.zeros((len(qubits), len(edges)), dtype=object)

    # Create symbolic variables for edges
    edge_symbols = [sp.symbols(f'x{i}') for i in range(len(edges))]

    # Apply special conditions for qubits
    conditions = [
        (lambda i, j: ((i % 2 == 0 and (j % 8 == 0 or j % 8 == 6)) or (i % 2 == 1 and (j % 8 == 2 or j % 8 == 4)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i - 1, j - 1), (i - 1, j)),
            ((i - 1, j), (i, j)),
            ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 1 and (j % 8 == 0 or j % 8 == 6)) or (i % 2 == 0 and (j % 8 == 2 or j % 8 == 4)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i + 1, j - 1), (i + 1, j)),
            ((i, j), (i + 1, j)),
             ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 0 and (j % 8 == 1 or j % 8 == 7)) or (i % 2 == 1 and (j % 8 == 3 or j % 8 == 5)), [
            ((i, j - 2), (i, j - 1)),
            ((i - 1, j - 1), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 1 and (j % 8 == 1 or j % 8 == 7)) or (i % 2 == 0 and (j % 8 == 3 or j % 8 == 5)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i + 1, j)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1))
        ]))
    ]


    print(edges)
    for f in conditions:
        for i in range(n):
            for j in range(m):
                condition, special_edges = f(i, j)
                if condition:
                    for (rel_i1, rel_j1), (rel_i2, rel_j2) in special_edges:
                        # Compute actual coordinates of edge
                        edge = ((rel_i1, rel_j1), (rel_i2, rel_j2))

                        # Ensure the edge exists before modifying the matrix
                        if edge in edges:
                            e_idx = edges[edge]
                            q_idx = qubits[(i, j)]
                            # Use symbolic variables for edges
                            matrix[q_idx, e_idx] = edge_symbols[e_idx]
                        else:
                            print(edge)

    # Print the edge mapping (index to actual edge)
    print("\nEdge to Column Mapping:")
    for edge, idx in edges.items():
        print(f"Column {idx} corresponds to edge: {edge}")

    print("\nQubit to Row Mapping:")
    for qubit, idx in qubits.items():
        print(f"Row {idx} corresponds to qubit: {qubit}")

    # Return matrix with symbolic edge variables
    return matrix, qubits, edges, edge_symbols

# Define grid size
n, m = 2, 8  # n must be even

# Generate the matrix with unknown edge parameters
qubit_edge_matrix, qubit_map, edge_map, edge_symbols = generate_qubit_edge_matrix_with_unknowns(n, m)


# Display symbolic edge variables
print("\nEdge Variables:")
for i, symbol in enumerate(edge_symbols):
    print(f"x{i}: {symbol}")

# Display result
print("Qubit-Edge Matrix (with symbolic edge parameters):")
print(qubit_edge_matrix)


{((0, 0), (0, 1)): 0, ((0, 1), (0, 2)): 1, ((0, 2), (0, 3)): 2, ((0, 3), (0, 4)): 3, ((0, 4), (0, 5)): 4, ((0, 5), (0, 6)): 5, ((0, 6), (0, 7)): 6, ((1, 0), (1, 1)): 7, ((1, 1), (1, 2)): 8, ((1, 2), (1, 3)): 9, ((1, 3), (1, 4)): 10, ((1, 4), (1, 5)): 11, ((1, 5), (1, 6)): 12, ((1, 6), (1, 7)): 13, ((0, 3), (1, 3)): 14, ((0, 5), (1, 5)): 15}
((0, -2), (0, -1))
((0, -1), (0, 0))
((-1, -1), (-1, 0))
((-1, 0), (0, 0))
((-1, 5), (-1, 6))
((-1, 6), (0, 6))
((0, 2), (1, 2))
((0, 4), (1, 4))
((0, 2), (1, 2))
((0, 4), (1, 4))
((1, -2), (1, -1))
((1, -1), (1, 0))
((2, -1), (2, 0))
((1, 0), (2, 0))
((2, 5), (2, 6))
((1, 6), (2, 6))
((0, -1), (0, 0))
((-1, 0), (0, 0))
((-1, 6), (0, 6))
((0, 7), (0, 8))
((0, 2), (1, 2))
((0, 4), (1, 4))
((0, 2), (1, 3))
((0, 4), (1, 5))
((1, -1), (1, 0))
((1, 0), (2, 1))
((1, 6), (2, 7))
((1, 7), (1, 8))

Edge to Column Mapping:
Column 0 corresponds to edge: ((0, 0), (0, 1))
Column 1 corresponds to edge: ((0, 1), (0, 2))
Column 2 corresponds to edge: ((0, 2), (0, 3